In [1]:
import numpy as np
import random
import pandas as pd
from random import randint
import math

In [2]:
class Particle_of_swarm(object):
    def __init__(self, hiddenSize, inputSize, outputSize):
        # initiate layers
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize

        layers = [self.inputSize] + self.hiddenSize + [self.outputSize]

        # initiate positions
        positions = []
        for i in range(len(layers)-1):
            p = np.random.uniform(-10,10,(layers[i], layers[i+1]))
            #p = np.random.rand(layers[i], layers[i+1])
            positions.append(p)
        self.positions = positions
        self.positions_best = positions

        velocitys = []
        for i in range(len(layers) - 1):
            v = np.random.uniform(-10,10,(layers[i], layers[i+1]))
            #v = np.random.rand(layers[i], layers[i+1])
            velocitys.append(v)
        self.velocitys = velocitys

        self.pbest = float('inf')

    def feedForward(self, X):
        Output_node = X
        for i, p in enumerate(self.positions):
            
            # print("i = ",i,len(self.positions))
            # print(Output_node.shape)
            # print(p.shape)
            # print("========================================")
            v = np.dot(Output_node, p)
            Output_node = self.sigmoid(v)

        return Output_node

    def sigmoid(self, s, deriv=False):
        if (deriv == True):
            return s * (1-s)
        return 1/(1 + np.exp(-s))

    def object_funct(self, X, Y):
                    # Random data
        seed = randint(1, 25*100)

        np.random.seed(seed)
        np.random.shuffle(X)

        np.random.seed(seed)
        np.random.shuffle(Y)

        sum_err = 0
        for j, input in enumerate(X):
            target = Y[j]
            output = self.feedForward(input)

            sum_err += self._mae(target, output)

        self.fx = sum_err/len(X) #(1/(sum_err+1))
        return self.fx

    def _mae(self, target, output):
        #print(target - output)
        return np.average(abs(target - output))



In [3]:
def _readfile(file):

    # ----- Clean NaN Values -----
    df = pd.read_csv(file)
    df = df.fillna(method = 'ffill')

    # ----- Create Features -----
    X = df[['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)','T','RH','AH']].copy(deep=False)
    X.drop(X.tail(240).index,inplace=True)
    X['PT08.S1(CO)'] = X['PT08.S1(CO)']/X['PT08.S1(CO)'].max()
    X['PT08.S2(NMHC)'] = X['PT08.S2(NMHC)']/X['PT08.S2(NMHC)'].max()
    X['PT08.S3(NOx)'] = X['PT08.S3(NOx)']/X['PT08.S3(NOx)'].max()
    X['PT08.S4(NO2)'] = X['PT08.S4(NO2)']/X['PT08.S4(NO2)'].max()
    X['PT08.S5(O3)'] = X['PT08.S5(O3)']/X['PT08.S5(O3)'].max()
    X['T'] = X['T']/X['T'].max()
    X['RH'] = X['RH']/X['RH'].max()
    X['AH'] = X['AH']/X['AH'].max()
    
    # ----- Create Desired outputs -----
    label = df[['C6H6(GT)']].copy(deep=False)

    Y_10Day = label.iloc[240:,:].reset_index(drop=True)
    Y_10Day.rename(columns={"C6H6(GT)":"C6H6(GT)_10"}, inplace = True)

    Y_5Day = label.iloc[120:,:].reset_index(drop=True)
    Y_5Day.drop(Y_5Day.tail(120).index,inplace=True)
    Y_5Day.rename(columns={"C6H6(GT)":"C6H6(GT)_5"}, inplace = True)

    Y = pd.concat([Y_5Day,Y_10Day], axis=1)
  
    Y['C6H6(GT)_5'] = Y['C6H6(GT)_5']/Y['C6H6(GT)_5'].max()
    
    Y["C6H6(GT)_10"] = Y["C6H6(GT)_10"]/Y["C6H6(GT)_10"].max()
    Input = X.to_numpy()
    #Output = Y.to_numpy()
    Output = Y['C6H6(GT)_5'].to_numpy()
    return Input,Output

Input,Output = _readfile('data/AirQualityUCI.csv')
Input,Output = np.genfromtxt('data/AirQualityUCI.csv', delimiter=',')
#print(Input)


In [4]:
def cross_validations_split(shape,folds):
    fold_size = int(shape * folds/100)
    k = 0
    index = []
    for i in range(1,folds+1):
        if i < folds:
            index.append([k,i*fold_size])
        else:
            index.append([k,shape])
        k = i*fold_size
    return index

In [5]:

particles = []
num_of_particle = 12


for i in range(0, num_of_particle):
    par = Particle_of_swarm([4], 8, 2)
    particles.append(par)


gbest = [0,float('inf')]
gbest_position = 0
#-------------------------------------------------------

train_mean_pbest = []
train_gbest = []

test_mean_pbest = []

#-------------------------------------------------------
k=1
for a,b in cross_validations_split(Input.shape[0],10):
    x_train = np.concatenate((Input[:a],Input[b+1:]))
    y_train = np.concatenate((Output[:a],Output[b+1:]))
    x_test = Input[a:b,:]
    y_test = Output[a:b]
    list_fx = []
    print("-------------------- fold : ",k," --------------------")
    k+=1
    for j in range(25):
        for i,p in enumerate(particles):
            fx = p.object_funct(x_train, y_train)
            list_fx.append(fx)

        # check pbest
            if fx < p.pbest:
                print("Update pbest ",i," : ",round(fx,3), " / ",round(p.pbest,3))
                p.pbest = fx
                p.positions_best = p.positions.copy()
                

        #check gbest
            if fx < gbest[1]:
                gbest[1] = fx
                gbest[0] = i
                gbest_position = p.positions.copy()
                print("Update gbest",i," : ",fx)

        # update velocity & position
        for i,p in enumerate(particles):
            for c in range(0, len(p.velocitys)):
                d1 = 2
                d2 = 3
                d = d1+d2
                g = 1 - (1/(d)) + math.sqrt(abs(d**2 - 4*d))/2
                p.velocitys[c] = g*(0.2*p.velocitys[c] + (d1 * (p.positions_best[c]-p.positions[c])) + (d2 * (gbest_position[c] - p.positions[c])))
                p.positions[c] += p.velocitys[c]


        print(" Epoch : " , j+1 ," | err : ", sum(list_fx)/len(list_fx) )

    train_mean_pbest.append(sum(list_fx)/len(list_fx))
    train_gbest.append(gbest[1])
    
    list_fx_test = []
    for i,p in enumerate(particles):
        fx = p.object_funct(x_test, y_test)
        list_fx_test.append(fx)
    test_mean_pbest.append(sum(list_fx_test)/len(list_fx_test))
    print("=========== Test ===========")
    print("Error of test  :  ",round(sum(list_fx_test)/len(list_fx_test),6))
    print("Error of train :  ",round(sum(list_fx)/len(list_fx),6))
    print("gbest : ",round(gbest[1],6))





x 10  :  0.3250618711835735
--- fx 11  :  0.5377748568137705
 Epoch :  16  | err :  0.39133555360143985
--- fx 0  :  0.1478326982232455
--- fx 1  :  0.5391750062421252
--- fx 2  :  0.25473019189394186
--- fx 3  :  0.7913274155112737
--- fx 4  :  0.16008627048062835
Update pbest  4  :  0.16  /  0.16
--- fx 5  :  0.8190414965029764
--- fx 6  :  0.5
--- fx 7  :  0.49993686125835524
--- fx 8  :  0.3159261735913608
--- fx 9  :  0.3648611856499034
--- fx 10  :  0.2572616727748616
--- fx 11  :  0.15992564934847436
Update pbest  11  :  0.16  /  0.16
 Epoch :  17  | err :  0.3918947593772236
--- fx 0  :  0.14783269822324552
--- fx 1  :  0.16010970834684568
--- fx 2  :  0.25473019189394164
--- fx 3  :  0.4756265324298754
--- fx 4  :  0.48917141881362297
--- fx 5  :  0.45150370653112903
--- fx 6  :  0.1742900000669646
--- fx 7  :  0.49980627886086243
--- fx 8  :  0.5660414017948617
--- fx 9  :  0.5
--- fx 10  :  0.3250618711835744
--- fx 11  :  0.5377748568137698
 Epoch :  18  | err :  0.39133555

KeyboardInterrupt: 

In [6]:
print(train_mean_pbest)
print(train_gbest)
print(test_mean_pbest)
result = pd.DataFrame({'train_average': train_mean_pbest, 'gbest': train_gbest,'validate_average': test_mean_pbest})

[0.159260464893184, 0.18134342186504046, 0.12229647166952977, 0.12370392290619572, 0.12186471812338497, 0.11670999058876413, 0.11930496918684148, 0.11950213708649476, 0.12491608406859406, 0.12408601745821815]
[0.1252334629015704, 0.12119056514576351, 0.12119056514576351, 0.12119056514576351, 0.12119056514576351, 0.11665406631069972, 0.11665406631069972, 0.11665406631069972, 0.11665406631069972, 0.11665406631069972]
[0.11139687905530593, 0.12912933906500038, 0.12155197511942827, 0.10248525669677493, 0.11905179264254782, 0.16613418447239542, 0.14234963411659948, 0.14037082032608514, 0.09158955297270066, 0.09951702155237664]


In [7]:
result.to_csv('swarm1.csv', encoding='utf-8')

In [8]:
for i,p in enumerate(particles):
    print(i,p.positions[1][2])

0 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
1 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
2 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
3 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
4 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
5 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
6 [-12.20571318  10.02295934   3.37011659 -11.94023421  14.65795875
   4.03513535 -16.11969517   0.19200045 -18.33861762   6.57171457]
7 [-12.20571318  10.02295934   3.37011659 -11.94023421 